In [1]:
import os

# Set environment variables
# OPENAI_API_KEYとLANGCHAIN_API_KEYは.envから読み込む
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "ai-agent-tutorial"

In [2]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。"
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。"
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。"
    }
}

In [3]:
import operator
from typing import Annotated

from pydantic import BaseModel, Field


class State(BaseModel):
    query: str = Field(..., description="ユーザーからの質問")
    current_role: str = Field(
        default="", description="選定された回答ロール"
    )
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    current_judge: bool = Field(
        default=False, description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="", description="品質チェックの判定理由"
    )

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# 後からmax_tokensの値を変更できるように、変更可能なフィールドを宣言
llm = llm.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [6]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> dict[str, Any]:
    query = state.query
    role_options = "\n".join([f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()])
    prompt = ChatPromptTemplate.from_template(
"""質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [7]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    role_details = "\n".join([f"- {v['name']}: {v['details']}" for v in ROLES.values()])
    prompt = ChatPromptTemplate.from_template(
"""あなたは{role}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_details}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"role": role, "role_details": role_details, "query": query})
    return {"messages": [answer]}

In [8]:
class Judgement(BaseModel):
    judge: bool = Field(default=False, description="判定結果")
    reason: str = Field(default="", description="判定理由")

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]
    prompt = ChatPromptTemplate.from_template(
"""以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {
        "current_judge": result.judge,
        "judgement_reason": result.reason
    }

In [9]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [10]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [11]:
# selectionノードから処理を開始
workflow.set_entry_point("selection")

In [12]:
# selectionノードからansweringノードへ
workflow.add_edge("selection", "answering")
# answeringノードからcheckノードへ
workflow.add_edge("answering", "check")

In [13]:
from langgraph.graph import END

# checkノードから次のノードへの遷移に条件付きエッジを定義
# state.current_judgeの値がTrueならENDノードへ、Falseならselectionノードへ
workflow.add_conditional_edges(
    "check",
    lambda state: state.current_judge,
    {True: END, False: "selection"}
)

In [14]:
compiled = workflow.compile()

In [15]:
initial_state = State(query="生成AIについて教えてください")
result = compiled.invoke(initial_state)

In [17]:
display(result)

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、動画などの生成が含まれます。生成AIの代表的な技術には、以下のようなものがあります。\n\n1. **GPT（Generative Pre-trained Transformer）**: これは、自然言語処理に特化したモデルで、テキストの生成や翻訳、要約などに利用されます。OpenAIのGPTシリーズが有名で、特にGPT-3やGPT-4は非常に高い精度で人間のようなテキストを生成することができます。\n\n2. **GAN（Generative Adversarial Networks）**: これは、画像生成に特化した技術で、二つのニューラルネットワーク（生成者と識別者）が競い合うことで、非常にリアルな画像を生成します。GANは、写真のような画像やアート作品の生成に利用されています。\n\n3. **VAE（Variational Autoencoders）**: これは、データの潜在的な構造を学習し、新しいデータを生成するために使用されます。VAEは、画像や音声の生成に利用されることが多いです。\n\n生成AIは、クリエイティブなコンテンツの制作、デザインの自動化、カスタマーサービスのチャットボット、教育コンテンツの生成など、さまざまな分野で活用されています。また、生成AIの進化により、より自然で人間らしいインタラクションが可能になり、ビジネスやエンターテインメントの分野での応用が広がっています。\n\nただし、生成AIには倫理的な課題も存在します。例えば、フェイクニュースの生成や著作権の問題などが挙げられます。これらの課題に対処するために、生成AIの開発と利用には慎重なアプローチが求められています。'],
 'current_judge': True,
 'judgement_reason': 'この回答は、生成AIについての基本的な情報を網羅的に提供しており、ユーザーの質問に対して適切に応答しています。以下の点で品質が高いと判断

In [18]:
print(result["messages"][-1])

生成AI製品エキスパートとしてお答えします。

生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、動画などの生成が含まれます。生成AIの代表的な技術には、以下のようなものがあります。

1. **GPT（Generative Pre-trained Transformer）**: これは、自然言語処理に特化したモデルで、テキストの生成や翻訳、要約などに利用されます。OpenAIのGPTシリーズが有名で、特にGPT-3やGPT-4は非常に高い精度で人間のようなテキストを生成することができます。

2. **GAN（Generative Adversarial Networks）**: これは、画像生成に特化した技術で、二つのニューラルネットワーク（生成者と識別者）が競い合うことで、非常にリアルな画像を生成します。GANは、写真のような画像やアート作品の生成に利用されています。

3. **VAE（Variational Autoencoders）**: これは、データの潜在的な構造を学習し、新しいデータを生成するために使用されます。VAEは、画像や音声の生成に利用されることが多いです。

生成AIは、クリエイティブなコンテンツの制作、デザインの自動化、カスタマーサービスのチャットボット、教育コンテンツの生成など、さまざまな分野で活用されています。また、生成AIの進化により、より自然で人間らしいインタラクションが可能になり、ビジネスやエンターテインメントの分野での応用が広がっています。

ただし、生成AIには倫理的な課題も存在します。例えば、フェイクニュースの生成や著作権の問題などが挙げられます。これらの課題に対処するために、生成AIの開発と利用には慎重なアプローチが求められています。


In [20]:
print(result["judgement_reason"])

この回答は、生成AIについての基本的な情報を網羅的に提供しており、ユーザーの質問に対して適切に応答しています。以下の点で品質が高いと判断できます。

1. **明確な定義**: 生成AIの定義が明確に示されており、どのようなコンテンツを生成できるかが具体的に説明されています。

2. **技術の紹介**: GPT、GAN、VAEといった代表的な生成AI技術が紹介され、それぞれの用途や特徴が簡潔に説明されています。

3. **応用例の提示**: 生成AIがどのような分野で活用されているか、具体的な例を挙げて説明しており、ユーザーが生成AIの実用性を理解しやすくなっています。

4. **倫理的課題の指摘**: 生成AIに関連する倫理的な課題についても触れており、技術の利点だけでなく、注意すべき点も示しています。

全体として、情報が整理されており、ユーザーが生成AIについての基本的な理解を得るのに十分な内容です。


In [21]:
initial_state = State(query="生成AIについて教えてください")
result = await compiled.ainvoke(initial_state)
result

{'query': '生成AIについて教えてください',
 'current_role': '生成AI製品エキスパート',
 'messages': ['生成AI製品エキスパートとしてお答えします。\n\n生成AI（生成的人工知能）は、データから新しいコンテンツを生成する能力を持つAI技術の一分野です。これには、テキスト、画像、音声、音楽など、さまざまな形式のコンテンツが含まれます。生成AIの代表的な技術には、以下のようなものがあります。\n\n1. **自然言語処理（NLP）モデル**: 例えば、GPT（Generative Pre-trained Transformer）シリーズは、テキストの生成や翻訳、要約などに利用されます。これらのモデルは、大量のテキストデータを学習し、人間のように自然な文章を生成することができます。\n\n2. **画像生成モデル**: GAN（Generative Adversarial Networks）やVAE（Variational Autoencoders）などの技術を用いて、新しい画像を生成します。これにより、アート作品の創作や、現実には存在しない人物の顔を生成することが可能です。\n\n3. **音声合成**: TTS（Text-to-Speech）技術を用いて、テキストから自然な音声を生成します。これにより、音声アシスタントやナレーションの自動生成が可能になります。\n\n生成AIは、クリエイティブなプロセスを支援し、効率を向上させるために多くの分野で活用されています。例えば、コンテンツ制作、ゲーム開発、広告、教育などです。しかし、生成AIの利用には倫理的な課題も伴います。例えば、偽情報の生成や著作権の問題などが挙げられます。これらの課題に対処するためには、技術の透明性や責任ある利用が求められています。\n\n生成AIは急速に進化しており、今後も新しい応用が期待されています。最新の研究や製品情報を追い続けることが重要です。'],
 'current_judge': True,
 'judgement_reason': 'この回答は、生成AIについての基本的な情報を網羅的に提供しており、ユーザーの質問に対して適切に応答しています。以下の点で品質が高いと判断できます。\n\n1. **明確な定義**: 生成AIの定義を簡潔に説

In [ ]:
!apt-get install graphviz libgraphviz-dev pkg-config
!pip install pygraphviz

In [ ]:
https://colab.research.google.com/github/GenerativeAgents/agent-book/blob/main/chapter09/notebook.ipynb#scrollTo=42mZywA6kzWl